In [1]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large-instruct")

/Users/peerasit/senior_project/STELLA-Backend/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Prepare PDF FILE

#### Sematic chunking format markdown

In [2]:
# import pymupdf4llm
# doc2 = pymupdf4llm.to_markdown('../../data_sources/ba_56-1_2023.PDF', page_chunks=True)
# semantic_chunk = SemanticChunker(embeddings=embedding_model, breakpoint_threshold_type="percentile")
# text_chunks = semantic_chunk.create_documents([d["text"] for d in doc2])

#### Semantic Chunking text

In [3]:
# import pymupdf
# from langchain_core.documents import Document

# doc = pymupdf.open("../../chunking/pdfs/bts_56-1_2023.pdf")
# semantic_chunker = SemanticChunker(embeddings=embedding_model, breakpoint_threshold_type="percentile")
# text_chunks = semantic_chunker.create_documents([d.get_text() for d in doc])

#### Hybrid Chunking (Structure Chunking + Semantic Chunking)

In [4]:
import sys
sys.path.insert(0, "/Users/peerasit/senior_project/STELLA-Backend/")
from chunking.one_report_file import oneReportFileChunking

text_chunks = oneReportFileChunking(content="../../chunking/pdfs/bts_56-1_2023.pdf", file_name="bts_56-1_2023.pdf")

## STELLA CORE DB

In [5]:
import sys
# from pymilvus import db
# sys.path.insert(1, '../../milvus_core/')
sys.path.insert(0, "/Users/peerasit/senior_project/STELLA-Backend")
sys.path.insert(0, "/Users/peerasit/senior_project/STELLA-Backend/milvus")

from milvus.core import Core
from milvus.schema import INDEX_PARAMS, DATA_SOURCE_SCHEMA

# core = Core(schema=DATA_SOURCE_SCHEMA, dense_embedding_model=embedding_model, collection_name="semantic", system_prune=True, database_name="semantic_ragas")
core = Core(
            database_name="semantic_ragas",
            schema=DATA_SOURCE_SCHEMA,
            dense_embedding_model=embedding_model,
            create_first_node=True,
            system_prune_first_node=True,
            token=""
        )

# core.add_document(text_chunks)

[CORE] Initializing Milvus Database Core...
[DB] init Embedding Model...
[DB] init Embedding Model Successfully.
[DB] Found Database: semantic_ragas
[DB] Found Collection "cnode_1".
[DB] Drop Collection "cnode_1"...
cnode_1 has: 49 entities
[DB] Drop Collection "cnode_1" Successfully.
[DB] Create Collection "cnode_1"
[DB] Collection "cnode_1" Is Ready.
[DB] Found Collection "gnode_1".
[DB] Drop Collection "gnode_1"...
gnode_1 has: 0 entities
[DB] Drop Collection "gnode_1" Successfully.
[DB] Create Collection "gnode_1"
[DB] Collection "gnode_1" Is Ready.
[DB] Found Collection "frontend_query_general_documents".
[DB] Drop Collection "frontend_query_general_documents"...
frontend_query_general_documents has: 0 entities
[DB] Drop Collection "frontend_query_general_documents" Successfully.
[DB] Create Collection "frontend_query_general_documents"
[DB] Collection "frontend_query_general_documents" Is Ready.
Create Schma Successfuly.


In [6]:
core.add_document(name="bts", documents=text_chunks, node_type="c", sector_id=20)

<Collection>:
-------------
<name>: cnode_1
<description>: Schema for Data Source Collection
<schema>: {'auto_id': True, 'description': 'Schema for Data Source Collection', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'dense_vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1024}}, {'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 8192}}, {'name': 'metadata', 'description': '', 'type': <DataType.JSON: 23>}], 'enable_dynamic_field': True}

<class 'pymilvus.orm.collection.Collection'>
[DB] Commit new Company
[DB] Create New Partition
[DB] Partition bts: 38 entities


In [113]:
# chunk_retriever_vectorStore = core.initVectorStore()
# chunk_retriever = chunk_retriever_vectorStore.as_retriever(search_kwargs={"k" : 1})
config ={
    "k": 1,
    "partition_names": ["bts"],
}
chunk_retriver = core.initVectorStore(collection_name="cnode_1", partition_names=["bts"], search_kwargs=config)

In [114]:
chunk_retriver.invoke("องประกอบของคณะกรรมการบริษัท")

[Document(metadata={'file_name': 'bts_56-1_2023.pdf', 'company_name': 'BTS', 'file_type': '56-1', 'year': '2023', 'structured': True, 'section': 'โครงสร้างการกำกับดูแลกิจการ\nและข้อมูลสำคัญเกี่ยวกับ\nคณะกรรมการ คณะกรรมการชุดย่อย ผู้บริหาร\nพนักงานและอื่นๆ', 'sub_section': 'ข้อมูลเกี่ยวกับคณะกรรมการ (7.2)'}, page_content='ข้อมูลเกี่ยวกับคณะกรรมการ (7.2)\nองค์ประกอบของคณะกรรมการบริษัท (7.2.1)\nจำนวน (คน) ร้อยละ (%)\nกรรมการรวม 11 100.00\nกรรมการชาย 10 90.91\nกรรมการหญิง 1 9.09\nกรรมการที่เป็นผู้บริหาร 4 36.36\nกรรมการที่ไม่เป็นผู้บริหาร 7 63.64\nกรรมการอิสระ 6 54.55\nข้อมูลคณะกรรมการและผู้มีอำนาจควบคุมบริษัทรายบุคคล (7.2.2)\nรายชื่อกรรมการชุดปัจจุบัน\nข้อมูลทั่วไป:\n 1. นาย คีรี กาญจนพาสน์\nเพศ: ชาย\nอายุ: 74 ปี\nวุฒิการศึกษา: ต่ำกว่าปริญญาตรี\nสาขา: -\nสัญชาติไทย: ใช่\nถิ่นที่อยู่ในประเทศไทย: ใช่\n\nตำแหน่งของกรรมการ:\n ประธานกรรมการ\n(กรรมการที่ไม่เป็นผู้บริหาร)\nประเภทของกรรมการ: กรรมการ\nเดิม\n\nวันเริ่มดำรงตำแหน่ง:\n 18 ธ.ค. 2549\n\nประสบการณ์และความชำนาญ:\n พัฒนาอสังหาริมทรัพย์, ขน

In [115]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"] = os.getenv("OPEN_AI_API_KEY")
llm = ChatOpenAI(model="gpt-3.5-turbo",temperature=0, max_tokens=4096)

# LLM RAG

In [116]:
from langchain_core.prompts import ChatPromptTemplate

rag_template = """\
Use the following context to answer the user's query. If you cannot answer, please respond with 'I don't know'.

User's Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(rag_template)

In [117]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo",temperature=0, max_tokens=4096)

recursive_rag_chain = (
    {"context" : chunk_retriver, "question" : RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [118]:
synthetic_data_chunks = text_chunks

# LLM Question

In [119]:
question_prompt = """\
You are a teacher preparing a test. Please create a question that can be answered by referencing the following context.

Context:
{context}
"""

question_prompt = ChatPromptTemplate.from_template(question_prompt)
question_chain = question_prompt | llm | StrOutputParser()

# LLM Ground Truth

In [120]:
ground_truth_prompt = """\
Use the following context and question to answer this question using *only* the provided context.

Question:
{question}

Context:
{context}
"""

ground_truth_prompt = ChatPromptTemplate.from_template(ground_truth_prompt)
ground_truth_chain = ground_truth_prompt | llm | StrOutputParser()

## Create Synthesized Data

In [121]:
questions = []
ground_truths_recursive = []
contexts = []
answers = []

for chunk in synthetic_data_chunks[0:48]:
    # print(chunk)
  questions.append(question_chain.invoke({"context" : chunk.page_content}))
  contexts.append(chunk.page_content)
  ground_truths_recursive.append(ground_truth_chain.invoke({"question" : questions[-1], "context" : str(contexts[-1])}))
  
  answers.append(recursive_rag_chain.invoke(questions[-1]))

# Create Synthesized Data for Evaluation

In [122]:
from datasets import load_dataset, Dataset

qagc_list = []

for question, answer, context, ground_truth in zip(questions, answers, contexts, ground_truths_recursive):
  qagc_list.append({
      "question" : question,
      "answer" : answer,
      "contexts" : [context],
      "ground_truth" : ground_truth
  })

eval_dataset = Dataset.from_list(qagc_list)
eval_dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 38
})

In [123]:
eval_dataset[13]


{'question': 'Question: What is the educational background of Mr. กวิน กาญจนพาสน์?',
 'answer': "Mr. กวิน กาญจนพาสน์ has an educational background that is lower than a bachelor's degree.",
 'contexts': ['ข้อมูลทั่วไป:\n 5. นาย กวิน กาญจนพาสน์\nเพศ: ชาย\nอายุ: 49 ปี\nวุฒิการศึกษา: ต่ำกว่าปริญญาตรี\nสาขา: -\nสัญชาติไทย: ใช่\nถิ่นที่อยู่ในประเทศไทย: ใช่\n\nตำแหน่งของกรรมการ:\n กรรมการ\n(กรรมการที่เป็นผู้บริหาร)\nประเภทของกรรมการ: กรรมการ\nที่หมดวาระและได้รับการแต่งตั้ง\nกลับเข้ามาใหม่\n\nวันเริ่มดำรงตำแหน่ง:\n 29 ก.ค. 2553\n\nประสบการณ์และความชำนาญ:\n ประกันภัยและประกันชีวิต, พัฒนา\nอสังหาริมทรัพย์, ขนส่งและ\nโลจิสติกส์, สื่อและสิ่งพิมพ์,\nเทคโนโลยีสารสนเทศและการ\nสื่อสาร, การตลาด, การเงิน, การ\nตลาดดิจิทัล, วิศวกรรม, การจัดการ\nองค์กร, ผู้นำ, การจัดการกลยุทธ์,\nการจัดการความเสี่ยง\nข้อมูลทั่วไป:\n 6. นาย สุรพงษ์ เลาหะอัญญา\nเพศ: ชาย\nอายุ: 62 ปี\nวุฒิการศึกษา: ปริญญาโท\nสาขา: วิศวกรรม\nสัญชาติไทย: ใช่\nถิ่นที่อยู่ในประเทศไทย: ใช่\n\nตำแหน่งของกรรมการ:\n กรรมการ\n(กรรมการที่เป็นผู้บริหาร)

# Evaluation with RAGAS

In [124]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

from ragas import evaluate

result = evaluate(
    eval_dataset,
    metrics=[
        context_precision,
        context_recall,
        # faithfulness,
        # answer_relevancy,
    ],
)
result

Evaluating: 100%|██████████| 76/76 [00:12<00:00,  5.98it/s]


{'context_precision': 0.9737, 'context_recall': 0.9020}

In [125]:
results_df = result.to_pandas()
results_df

,user_input,retrieved_contexts,response,reference,context_precision,context_recall
0,Question:\nWhat is the full name of the compan...,[นโยบายและภาพรวมการประกอบธุรกิจ (1.1)\nข้อมูลท...,The full name of the company mentioned in the ...,The full name of the company mentioned in the ...,1.0,1.000000
1,Question: What is the percentage of revenue fr...,[ลักษณะการประกอบธุรกิจ (1.2)\nโครงสร้างรายได้ ...,I don't know.,The percentage of revenue from the MOVE catego...,1.0,1.000000
2,Question: What are some of the factors that po...,[ปัจจัยความเสี่ยงต่อการดำเนินธุรกิจของบริษัท (...,Some of the factors that pose risks to the bus...,Some of the factors that pose risks to the bus...,1.0,0.777778
3,Question: What are some examples of operationa...,[ผลกระทบต่อสิ่งแวดล้อม • อื่น ๆ ความเสี่ยงจากก...,Examples of operational risks mentioned in the...,Some examples of operational risks mentioned i...,1.0,1.000000
4,Question: What are the main categories of risk...,[ความเสี่ยงจากการยกเลิกสัญญาสัมปทานและสัญญาการ...,The main categories of risk discussed in the c...,The main categories of risk discussed in the c...,1.0,1.000000
5,Question: What is the term used to describe th...,[สัดส่วนการถือหุ้นของผู้ลงทุน (Control Dilutio...,The term used to describe the proportion of st...,สัดส่วนการถือหุ้น (Stock ownership proportion),1.0,1.000000
6,Question: What is the URL for the sustainabili...,[นโยบายและเป้าหมายการจัดการด้านความยั่งยืน (3....,URL นโยบายด้านความยั่งยืนของบริษัท: https://ww...,URL for the sustainability policy of BTS Group...,1.0,1.000000
7,Question: What is the total amount of diesel f...,[การจัดการด้านความยั่งยืนในมิติสิ่งแวดล้อม (3....,I don't know.,The total amount of diesel fuel used by the co...,1.0,1.000000
8,"Question: According to the context provided, w...",[การจัดการความยั่งยืนในมิติสังคม (3.4)\nนโยบาย...,"According to the context provided, the key are...","According to the context provided, some of the...",1.0,1.000000
9,Question: What is the URL for the corporate go...,[ภาพรวมของนโยบายและแนวปฏิบัติการกำกับดูแลกิจกา...,URL for the corporate governance policy and co...,URL for the corporate governance policy and co...,1.0,1.000000


In [126]:
eval_dataset[35]

{'question': 'What is the position held by นาย คีรี กาญจนพาสน์ in the context provided?',
 'answer': 'นาย คีรี กาญจนพาสน์ holds the position of ประธานกรรมการบริหาร (Chairman of the Board) in the context provided.',
 'contexts': ['นาย การุญ จันทรางศุ กรรมการ วันที่ดำรงตำแหน่ง: พัฒนาอสังหาริมทรัพย์, ขนส่งและ\nเพศ: ชาย (กรรมการที่ไม่เป็นผู้บริหาร, 23 ก.ค. 2561 โลจิสติกส์, เทคโนโลยีสารสนเทศ\nอายุ: 75 ปี กรรมการอิสระ) และการสื่อสาร, วิศวกรรม, การ\nวุฒิการศึกษา: ปริญญาเอก จัดการความเสี่ยง\nสาขา: วิศวกรรม ประเภทของกรรมการ: กรรมการ\nสัญชาติไทย: ใช่ ที่หมดวาระและได้รับการแต่งตั้ง\nถิ่นที่อยู่ในประเทศไทย: ใช่ กลับเข้ามาใหม่\n26[END_PAGE]การพัฒนากรรมการในรอบปีที่ผ่านมา\nรายชื่อกรรมการ ตำแหน่งของกรรมการ เข้าร่วมการอบรมพัฒนากรรมการ\nนาย คีรี กาญจนพาสน์ ประธานกรรมการ เข้าร่วม\nนาย รังสิน กฤตลักษณ์ กรรมการ เข้าร่วม\nนาย ชิ เคือง คง กรรมการ เข้าร่วม\nนาย สุจินต์ หวั่งหลี กรรมการ เข้าร่วม\nนาย กวิน กาญจนพาสน์ กรรมการ เข้าร่วม\nนาย สุรพงษ์ เลาหะอัญญา กรรมการ เข้าร่วม\nนาย เจริญ วรรธนะสิน กรรมการ เข้าร่ว

In [127]:
results_df['context_precision'].mean()

0.9736842104289472

In [128]:
results_df['context_recall'].mean()

0.902046783625731